In [165]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import regex as re
from collections import defaultdict

things to look at: how accurate is the prematch probability when it ends (if a team has over 50% chance to win, how often do they actually win the game?)

need more data from other sportsbooks to calculate fair probability for spread markets 

In [166]:
date = '2025-11-22'
odds_sport = 'cfb' #cbb, cfb, nba, nfl
kalshi_sport = 'ncaaf' #ncaab, ncaaf, nba, nfl

In [167]:
odds_df = pd.read_csv(f"../data_collection/updated_scripts/oddsapi_outputs/{date}/{odds_sport}_odds.csv")
odds_df.drop(columns=['league'], inplace=True)
odds_df.rename(columns={'price': 'odds'}, inplace=True)
odds_df['vig_prob'] = 1 / odds_df['odds']

def remove_vig_probs(df):
    df = df.copy()
    df['fair_prb'] = pd.NA

    grouped = df.groupby(['game_id', 'bookmaker', 'market'])

    for _, group in grouped:
        if len(group) < 2:
            continue
        probs = group['vig_prob']
        total = probs.sum()
        if total == 0:
            continue
        fair_probs = (probs / total).round(4)
        df.loc[group.index, 'fair_prb'] = fair_probs

    return df

odds_df = remove_vig_probs(odds_df)


odds_winners_df = odds_df[odds_df['market'] == 'h2h'].copy()
odds_spreads_df = odds_df[odds_df['market'] == 'spreads'].copy()
odds_spreads_df = odds_spreads_df.loc[(odds_spreads_df['point'].notna()) & (odds_spreads_df['point'] > 0)]
odds_totals_df  = odds_df[odds_df['market'] == 'totals'].copy()

# Average per-team fair probabilities across DraftKings/FanDuel/Pinnacle for winners_df
mask = odds_winners_df['fair_prb'].notna()
avg_by_team = (
    odds_winners_df.loc[mask]
    .groupby(['game_id', 'team'])['fair_prb']
    .transform('mean')
    .round(4)
)
odds_winners_df.loc[mask, 'avg_fair_prb'] = avg_by_team
odds_winners_df.loc[~mask, 'avg_fair_prb'] = pd.NA

#Average fair probabilities for spreads for same game, point spread, and team
mask = odds_spreads_df['fair_prb'].notna()
avg_by_point = (
    odds_spreads_df.loc[mask]
    .groupby(['game_id', 'point', 'team'])['fair_prb']
    .transform('mean')
    .round(4)
)
odds_spreads_df['avg_fair_prb'] = avg_by_point

In [168]:
kalshi_winners_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_winners.csv")
kalshi_totals_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_totals.csv")
kalshi_spreads_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_spreads.csv")

if kalshi_sport == 'ncaaf':
    kalshi_spreads_df['points'] = kalshi_spreads_df['title'].str.extract(r'over ([\d.]+) points\?').astype(float)
elif kalshi_sport == 'ncaab':
    kalshi_spreads_df['points'] = kalshi_spreads_df['title'].str.extract(r'over ([\d.]+) Points\?').astype(float)


columns_to_drop = ['timestamp', 'market_type', 'yes_bid2', 'yes_ask2', 'no_bid2', 'no_ask2', 'yes_depth_bids', 'yes_depth_asks', 'no_depth_bids', 'no_depth_asks']
kalshi_winners_df.drop(columns=columns_to_drop, inplace=True)
kalshi_spreads_df.drop(columns=columns_to_drop, inplace=True)
kalshi_totals_df.drop(columns=columns_to_drop, inplace=True)

In [169]:
kalshi_spreads_df.head()

,ticker,title,status,event_start_time,yes_bid,yes_ask,no_bid,no_ask,yes_spread,no_spread,liquidity_dollars,volume_24h,points
0,KXNCAAFSPREAD-25NOV22FURCLEM-CLEM27,Clemson wins by over 27.5 points?,active,2025-12-06T16:30:00-05:00,0.52,0.98,0.02,0.48,0.46,0.46,499176.29,737.0,27.5
1,KXNCAAFSPREAD-25NOV22MERAUB-AUB27,Auburn wins by over 27.5 points?,active,2025-12-06T14:00:00-05:00,0.31,0.93,0.07,0.69,0.62,0.62,201823.88,8638.0,27.5
2,KXNCAAFSPREAD-25NOV22MERAUB-AUB23,Auburn wins by over 23.5 points?,active,2025-12-06T14:00:00-05:00,0.42,0.99,0.01,0.58,0.57,0.57,432593.72,9430.0,23.5
3,KXNCAAFSPREAD-25NOV22MERAUB-AUB20,Auburn wins by over 20.5 points?,active,2025-12-06T14:00:00-05:00,0.49,0.99,0.01,0.51,0.50,0.50,485583.32,1985.0,20.5
4,KXNCAAFSPREAD-25NOV22MERAUB-AUB17,Auburn wins by over 17.5 points?,active,2025-12-06T14:00:00-05:00,0.56,0.99,0.01,0.44,0.43,0.43,438927.60,4087.0,17.5


In [170]:
#get names from kalshi_winners_df
def extract_teams_from_winners(title):
    title = title.replace(" Winner?", "")
    if " at " in title:
        right, left = title.split(" at ", 1)
    else:
        return pd.Series([None, None])  
    left = re.sub(r'\bSt\.$', 'St', left.strip())
    right = re.sub(r'\bSt\.$', 'St', right.strip())
    return pd.Series([left, right])

kalshi_winners_df[['home_team', 'away_team']] = kalshi_winners_df['title'].apply(extract_teams_from_winners)
unique_rows = kalshi_winners_df.drop_duplicates(subset=['home_team', 'away_team'])
flat_teams = pd.unique(unique_rows[['home_team', 'away_team']].values.ravel())
kalshi_winners_teams = flat_teams.tolist()

#get names from kalshi_totals_df
def extract_team_from_totals(title):
    title = title.replace(": Total Points", "")
    if " at " in title:
        left = title.split(" at ", 1)[0].strip()
        left = re.sub(r'\bSt\.$', 'St', left)
        return left
    return None

kalshi_totals_df['away_team'] = kalshi_totals_df['title'].apply(extract_team_from_totals)
kalshi_totals_teams = kalshi_totals_df['away_team'].dropna().drop_duplicates().tolist()

#get names from kalshi_spreads_df
def extract_team_from_spreads(title):
    if " wins by " in title:
        team = title.split(" wins by ", 1)[0].strip()
        team = re.sub(r'\bSt\.$', 'St', team)
        return team
    return None

kalshi_spreads_df['team'] = kalshi_spreads_df['title'].apply(extract_team_from_spreads)
unique_teams_spread = kalshi_spreads_df['team'].drop_duplicates()
kalshi_spreads_teams = unique_teams_spread.tolist()

In [171]:
odds_teams_by_market = odds_df.groupby('market')['team'].unique().to_dict()

def fuzzy_match_kalshi_to_odds(kalshi_teams, odds_team_names):
    matched_kalshi = []
    matched_odds = []

    kalshi_sorted = sorted(kalshi_teams, key=lambda x: x[0] if x else '')
    remaining_odds = sorted(odds_team_names.tolist().copy(), reverse=True)

    for kalshi_name in kalshi_sorted:
        candidates = []
        for odds_name in remaining_odds:
            if kalshi_name in odds_name:
                candidates.append(odds_name)
        if len(candidates) == 1:
            matched_kalshi.append(kalshi_name)
            matched_odds.append(candidates[0])

    return matched_kalshi, matched_odds

matched_data = {}

# Winners / h2h
matched_kalshi_h2h, matched_odds_h2h = fuzzy_match_kalshi_to_odds(
    kalshi_winners_teams,
    odds_teams_by_market.get('h2h', [])
)

# Spreads
matched_kalshi_spreads, matched_odds_spreads = fuzzy_match_kalshi_to_odds(
    kalshi_spreads_teams,
    odds_teams_by_market.get('spreads', [])
)

# Totals (match only Over/Under)
totals_odds_df = odds_df[odds_df['market'] == 'totals']
odds_totals_teams = pd.unique(totals_odds_df[['home_team', 'away_team']].values.ravel())
matched_kalshi_totals, matched_odds_totals = fuzzy_match_kalshi_to_odds(
    kalshi_totals_teams,
    odds_totals_teams
)

matched_names = {
    'h2h': {
        'kalshi': matched_kalshi_h2h,
        'odds': matched_odds_h2h
    },
    'spreads': {
        'kalshi': matched_kalshi_spreads,
        'odds': matched_odds_spreads
    },
    'totals': {
        'kalshi': matched_kalshi_totals,
        'odds': matched_odds_totals
    }
}


In [172]:
assert(len(matched_names['h2h']['kalshi']) == len(matched_names['h2h']['odds']))
assert(len(matched_names['spreads']['kalshi']) == len(matched_names['spreads']['odds']))
assert(len(matched_names['totals']['kalshi']) == len(matched_names['totals']['odds']))

In [173]:
print("Missed teams in SPREAD market:\n")
for n in kalshi_spreads_teams:
    if n not in matched_names['spreads']['kalshi']:
        print(f'{n}\n')

Missed teams in SPREAD market:

New Mexico

Washington

Colorado

Tennessee

Florida

Illinois

Utah

Houston

Michigan

Kentucky

Duke

Texas



In [174]:
odds_winners_df = odds_winners_df[
    odds_winners_df['home_team'].isin(matched_names['h2h']['odds']) |
    odds_winners_df['away_team'].isin(matched_names['h2h']['odds'])
].drop_duplicates(subset='team').reset_index(drop=True)

kalshi_winners_df = kalshi_winners_df[
    kalshi_winners_df['home_team'].isin(matched_names['h2h']['kalshi']) |
    kalshi_winners_df['away_team'].isin(matched_names['h2h']['kalshi'])
].reset_index(drop=True)

odds_spreads_df = odds_spreads_df[odds_spreads_df['team'].isin(matched_names['spreads']['odds'])].reset_index(drop=True)
kalshi_spreads_df = kalshi_spreads_df[kalshi_spreads_df['team'].isin(matched_names['spreads']['kalshi'])].reset_index(drop=True)

odds_totals_df = odds_totals_df[
    odds_totals_df['home_team'].isin(matched_names['totals']['odds']) |
    odds_totals_df['away_team'].isin(matched_names['totals']['odds'])
].reset_index(drop=True)
kalshi_totals_df = kalshi_totals_df[kalshi_totals_df['away_team'].isin(matched_names['totals']['kalshi'])].reset_index(drop=True)


In [175]:
# Specify the columns to extract
kalshi_cols = ['ticker', 'yes_bid', 'yes_ask', 'home_team', 'away_team']
odds_cols = ['market', 'start_time', 'team', 'home_team', 'away_team', 'avg_fair_prb']

# Rename overlapping columns in odds to prevent clashes
odds_subset = odds_winners_df[odds_cols].rename(columns={
    'home_team': 'odds_home_team',
    'away_team': 'odds_away_team'
})

kalshi_subset = kalshi_winners_df[kalshi_cols].rename(columns={
    'home_team': 'kalshi_home_team',
    'away_team': 'kalshi_away_team'
})

seen_tickers = set()
seen_rows = {}
combined_rows = []

# Loop through Kalshi rows
for _, kalshi_row in kalshi_subset.iterrows():
    kalshi_home = kalshi_row['kalshi_home_team']
    for _, odds_row in odds_subset.iterrows():
        odds_home = odds_row['odds_home_team']
        if kalshi_home in odds_home:
            ticker = kalshi_row['ticker']
            if ticker not in seen_tickers:
                seen_tickers.add(ticker)
                seen_rows[ticker] = odds_row
                continue
            else:
                curr_prb = odds_row['avg_fair_prb']
                prev_prb = seen_rows[ticker]['avg_fair_prb']
                midpoint = (kalshi_row['yes_bid'] + kalshi_row['yes_ask']) / 2
                if ((curr_prb - midpoint) ** 2) < ((prev_prb - midpoint) ** 2):
                    combined_row = pd.concat([kalshi_row, odds_row])
                else:
                    combined_row = pd.concat([kalshi_row, seen_rows[ticker]])
                combined_rows.append(combined_row)
                break

combined_winners_df = pd.DataFrame(combined_rows)
combined_winners_df = combined_winners_df.reset_index(drop=True)

In [176]:
combined_winners_df

,ticker,yes_bid,yes_ask,kalshi_home_team,kalshi_away_team,market,start_time,team,odds_home_team,odds_away_team,avg_fair_prb
0,KXNCAAFGAME-25NOV22USMUSA-USM,0.14,0.15,South Alabama,Southern Miss,h2h,2025-11-22 14:32:18 CST,Southern Mississippi Golden Eagles,South Alabama Jaguars,Southern Mississippi Golden Eagles,0.486050
1,KXNCAAFGAME-25NOV22USMUSA-USA,0.85,0.86,South Alabama,Southern Miss,h2h,2025-11-22 14:32:18 CST,South Alabama Jaguars,South Alabama Jaguars,Southern Mississippi Golden Eagles,0.513950
2,KXNCAAFGAME-25NOV22WASHUCLA-WASH,0.78,0.79,UCLA,Washington,h2h,2025-11-22 21:30:00 CST,Washington Huskies,UCLA Bruins,Washington Huskies,0.775400
3,KXNCAAFGAME-25NOV22WASHUCLA-UCLA,0.22,0.23,UCLA,Washington,h2h,2025-11-22 21:30:00 CST,UCLA Bruins,UCLA Bruins,Washington Huskies,0.224600
4,KXNCAAFGAME-25NOV22TULNTEM-TULN,0.83,0.84,Temple,Tulane,h2h,2025-11-22 14:45:00 CST,Tulane Green Wave,Temple Owls,Tulane Green Wave,0.718467
...,...,...,...,...,...,...,...,...,...,...,...
73,KXNCAAFGAME-25NOV22NEVWYO-NEV,0.63,0.65,Wyoming,Nevada,h2h,2025-11-22 13:07:33 CST,Nevada Wolf Pack,Wyoming Cowboys,Nevada Wolf Pack,0.686067
74,KXNCAAFGAME-25NOV22EIUALA-EIU,0.00,0.01,Alabama,Eastern Illinois,h2h,2025-11-22 14:32:18 CST,Southern Mississippi Golden Eagles,South Alabama Jaguars,Southern Mississippi Golden Eagles,0.486050
75,KXNCAAFGAME-25NOV22EIUALA-ALA,0.99,1.00,Alabama,Eastern Illinois,h2h,2025-11-22 14:32:18 CST,South Alabama Jaguars,South Alabama Jaguars,Southern Mississippi Golden Eagles,0.513950
76,KXNCAAFGAME-25NOV22MOSUKENN-MOSU,0.26,0.28,Kennesaw St,Missouri St,h2h,2025-11-22 13:06:42 CST,Missouri State Bears,Kennesaw State Owls,Missouri State Bears,0.357800


In [177]:
combined_winners_df.loc[(combined_winners_df['avg_fair_prb'] > combined_winners_df['yes_ask'] + 0.02) |
                        (combined_winners_df['avg_fair_prb'] < combined_winners_df['yes_bid'] - 0.02)].reset_index(drop=True)

,ticker,yes_bid,yes_ask,kalshi_home_team,kalshi_away_team,market,start_time,team,odds_home_team,odds_away_team,avg_fair_prb
0,KXNCAAFGAME-25NOV22USMUSA-USM,0.14,0.15,South Alabama,Southern Miss,h2h,2025-11-22 14:32:18 CST,Southern Mississippi Golden Eagles,South Alabama Jaguars,Southern Mississippi Golden Eagles,0.486050
1,KXNCAAFGAME-25NOV22USMUSA-USA,0.85,0.86,South Alabama,Southern Miss,h2h,2025-11-22 14:32:18 CST,South Alabama Jaguars,South Alabama Jaguars,Southern Mississippi Golden Eagles,0.513950
2,KXNCAAFGAME-25NOV22TULNTEM-TULN,0.83,0.84,Temple,Tulane,h2h,2025-11-22 14:45:00 CST,Tulane Green Wave,Temple Owls,Tulane Green Wave,0.718467
3,KXNCAAFGAME-25NOV22TULNTEM-TEM,0.16,0.20,Temple,Tulane,h2h,2025-11-22 14:45:00 CST,Temple Owls,Temple Owls,Tulane Green Wave,0.281533
4,KXNCAAFGAME-25NOV22UKVAN-VAN,0.93,0.94,Vanderbilt,Kentucky,h2h,2025-11-22 10:59:00 CST,Vanderbilt Commodores,Vanderbilt Commodores,Kentucky Wildcats,0.760850
5,KXNCAAFGAME-25NOV22UKVAN-UK,0.07,0.08,Vanderbilt,Kentucky,h2h,2025-11-22 10:59:00 CST,Kentucky Wildcats,Vanderbilt Commodores,Kentucky Wildcats,0.239150
6,KXNCAAFGAME-25NOV22KSUUTAH-UTAH,0.76,0.77,Utah,Kansas St,h2h,2025-11-22 15:00:00 CST,Utah Utes,Utah Utes,Kansas State Wildcats,0.861900
7,KXNCAAFGAME-25NOV22KSUUTAH-KSU,0.24,0.25,Utah,Kansas St,h2h,2025-11-22 15:00:00 CST,Kansas State Wildcats,Utah Utes,Kansas State Wildcats,0.138100
8,KXNCAAFGAME-25NOV22OKSTUCF-UCF,0.70,0.71,UCF,Oklahoma St,h2h,2025-11-22 15:00:00 CST,UCF Knights,UCF Knights,Oklahoma State Cowboys,0.816533
9,KXNCAAFGAME-25NOV22OKSTUCF-OKST,0.29,0.31,UCF,Oklahoma St,h2h,2025-11-22 15:00:00 CST,Oklahoma State Cowboys,UCF Knights,Oklahoma State Cowboys,0.183467


In [178]:
odds_spreads_df

,sport,game_id,start_time,bookmaker,market,team,odds,point,home_team,away_team,vig_prob,fair_prb,avg_fair_prb
0,CFB,946ab16f8cfe9004a0d3fc67ac8411d7,2025-11-22 10:51:00 CST,DraftKings,spreads,Arkansas Razorbacks,1.87,8.5,Texas Longhorns,Arkansas Razorbacks,0.534759,0.5,0.491633
1,CFB,946ab16f8cfe9004a0d3fc67ac8411d7,2025-11-22 10:51:00 CST,FanDuel,spreads,Arkansas Razorbacks,1.88,8.5,Texas Longhorns,Arkansas Razorbacks,0.531915,0.5,0.491633
2,CFB,946ab16f8cfe9004a0d3fc67ac8411d7,2025-11-22 10:51:00 CST,Pinnacle,spreads,Arkansas Razorbacks,1.99,8.5,Texas Longhorns,Arkansas Razorbacks,0.502513,0.4749,0.491633
3,CFB,31667911777e27d23d74adaf5109d04e,2025-11-22 12:00:00 CST,FanDuel,spreads,Georgia Southern Eagles,1.80,42.5,Georgia Southern Eagles,Old Dominion Monarchs,0.555556,0.5213,0.52065
4,CFB,31667911777e27d23d74adaf5109d04e,2025-11-22 12:00:00 CST,DraftKings,spreads,Georgia Southern Eagles,1.80,42.5,Georgia Southern Eagles,Old Dominion Monarchs,0.555556,0.52,0.52065
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,CFB,435c548cea0f26f2df00d9c9df0a7924,2025-11-22 21:30:00 CST,FanDuel,spreads,San Jose State Spartans,1.89,11.5,San Diego State Aztecs,San Jose State Spartans,0.529101,0.5052,0.5
67,CFB,435c548cea0f26f2df00d9c9df0a7924,2025-11-22 21:30:00 CST,Pinnacle,spreads,San Jose State Spartans,1.97,12.0,San Diego State Aztecs,San Jose State Spartans,0.507614,0.4936,0.4936
68,CFB,f0ad800d071b9335db5bd1c5634a28ca,2025-11-22 21:30:00 CST,FanDuel,spreads,UCLA Bruins,1.93,10.5,UCLA Bruins,Washington Huskies,0.518135,0.4948,0.492633
69,CFB,f0ad800d071b9335db5bd1c5634a28ca,2025-11-22 21:30:00 CST,DraftKings,spreads,UCLA Bruins,1.95,10.5,UCLA Bruins,Washington Huskies,0.512821,0.4895,0.492633


In [179]:
kalshi_cols = ['ticker', 'yes_bid', 'yes_ask', 'team', 'points']
odds_cols = ['market', 'start_time', 'team', 'home_team', 'away_team', 'avg_fair_prb', 'point']

odds_subset = odds_spreads_df[odds_cols].rename(columns={
    'home_team': 'odds_home_team',
    'away_team': 'odds_away_team',
    'team': 'odds_team'
})

kalshi_subset = kalshi_spreads_df[kalshi_cols]

combined_rows = []

for _, kalshi_row in kalshi_subset.iterrows():
    kalshi_home = kalshi_row['team']
    for _, odds_row in odds_subset.iterrows():
        odds_home = odds_row['odds_team']
        if (kalshi_home in odds_home) and (kalshi_row['points'] == odds_row['point']): 
            combined_row = pd.concat([kalshi_row, odds_row])
            combined_rows.append(combined_row)

combined_spreads_df = pd.DataFrame(combined_rows).drop_duplicates(subset='ticker') #only works because oddsapi only pulls odds 
combined_spreads_df = combined_spreads_df.reset_index(drop=True)                    #for only one point line for each bookmaker 



In [180]:
kalshi_spreads_df

,ticker,title,status,event_start_time,yes_bid,yes_ask,no_bid,no_ask,yes_spread,no_spread,liquidity_dollars,volume_24h,points,team
0,KXNCAAFSPREAD-25NOV22FURCLEM-CLEM27,Clemson wins by over 27.5 points?,active,2025-12-06T16:30:00-05:00,0.52,0.98,0.02,0.48,0.46,0.46,499176.29,737.0,27.5,Clemson
1,KXNCAAFSPREAD-25NOV22MERAUB-AUB27,Auburn wins by over 27.5 points?,active,2025-12-06T14:00:00-05:00,0.31,0.93,0.07,0.69,0.62,0.62,201823.88,8638.0,27.5,Auburn
2,KXNCAAFSPREAD-25NOV22MERAUB-AUB23,Auburn wins by over 23.5 points?,active,2025-12-06T14:00:00-05:00,0.42,0.99,0.01,0.58,0.57,0.57,432593.72,9430.0,23.5,Auburn
3,KXNCAAFSPREAD-25NOV22MERAUB-AUB20,Auburn wins by over 20.5 points?,active,2025-12-06T14:00:00-05:00,0.49,0.99,0.01,0.51,0.50,0.50,485583.32,1985.0,20.5,Auburn
4,KXNCAAFSPREAD-25NOV22MERAUB-AUB17,Auburn wins by over 17.5 points?,active,2025-12-06T14:00:00-05:00,0.56,0.99,0.01,0.44,0.43,0.43,438927.60,4087.0,17.5,Auburn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,KXNCAAFSPREAD-25NOV22ODUGASO-ODU10,Old Dominion wins by over 10.5 points?,active,2025-12-06T13:00:00-05:00,0.98,1.00,0.00,0.02,0.02,0.02,872996.22,3133.0,10.5,Old Dominion
481,KXNCAAFSPREAD-25NOV22ODUGASO-GASO7,Georgia Southern wins by over 7.5 points?,active,2025-12-06T13:00:00-05:00,0.00,0.01,0.99,1.00,0.01,0.01,1377972.92,300.0,7.5,Georgia Southern
482,KXNCAAFSPREAD-25NOV22ODUGASO-GASO6,Georgia Southern wins by over 6.5 points?,active,2025-12-06T13:00:00-05:00,0.00,0.01,0.99,1.00,0.01,0.01,1354957.91,300.0,6.5,Georgia Southern
483,KXNCAAFSPREAD-25NOV22ODUGASO-GASO3,Georgia Southern wins by over 3.5 points?,active,2025-12-06T13:00:00-05:00,0.00,0.10,0.90,1.00,0.10,0.10,1291758.14,1300.0,3.5,Georgia Southern


In [181]:
combined_spreads_df

,ticker,yes_bid,yes_ask,team,points,market,start_time,odds_team,odds_home_team,odds_away_team,avg_fair_prb,point
0,KXNCAAFSPREAD-25NOV22UNMAFA-AFA3,0.26,0.35,Air Force,3.5,spreads,2025-11-22 18:00:00 CST,Air Force Falcons,Air Force Falcons,New Mexico Lobos,0.503867,3.5
1,KXNCAAFSPREAD-25NOV22JVSTFIU-JVST6,0.16,0.47,Jacksonville St,6.5,spreads,2025-11-22 14:33:17 CST,Jacksonville State Gamecocks,Florida International Panthers,Jacksonville State Gamecocks,0.484000,6.5
2,KXNCAAFSPREAD-25NOV22JVSTFIU-JVST3,0.22,0.99,Jacksonville St,3.5,spreads,2025-11-22 14:33:17 CST,Jacksonville State Gamecocks,Florida International Panthers,Jacksonville State Gamecocks,0.510700,3.5
3,KXNCAAFSPREAD-25NOV22USUFRES-USU2,0.36,0.42,Utah St,2.5,spreads,2025-11-22 21:30:00 CST,Utah State Aggies,Fresno State Bulldogs,Utah State Aggies,0.488300,2.5
4,KXNCAAFSPREAD-25NOV22BYUCIN-CIN2,0.36,0.40,Cincinnati,2.5,spreads,2025-11-22 19:00:00 CST,Cincinnati Bearcats,Cincinnati Bearcats,BYU Cougars,0.489200,2.5
5,KXNCAAFSPREAD-25NOV22CALSTAN-STAN3,0.24,0.33,Stanford,3.5,spreads,2025-11-22 18:30:00 CST,Stanford Cardinal,Stanford Cardinal,California Golden Bears,0.485050,3.5
6,KXNCAAFSPREAD-25NOV22PITTGT-PITT2,0.40,0.41,Pittsburgh,2.5,spreads,2025-11-22 18:00:00 CST,Pittsburgh Panthers,Georgia Tech Yellow Jackets,Pittsburgh Panthers,0.494367,2.5
7,KXNCAAFSPREAD-25NOV22NEBPSU-NEB7,0.07,0.14,Nebraska,7.5,spreads,2025-11-22 18:00:00 CST,Nebraska Cornhuskers,Penn State Nittany Lions,Nebraska Cornhuskers,0.501350,7.5
8,KXNCAAFSPREAD-25NOV22TULNTEM-TEM9,0.03,0.36,Temple,9.5,spreads,2025-11-22 14:45:00 CST,Temple Owls,Temple Owls,Tulane Green Wave,0.500000,9.5
9,KXNCAAFSPREAD-25NOV22ECUUTSA-ECU7,0.04,0.13,East Carolina,7.5,spreads,2025-11-22 14:30:00 CST,East Carolina Pirates,UTSA Roadrunners,East Carolina Pirates,0.489300,7.5


In [182]:
odds_spreads_df.loc[odds_spreads_df['team'] == 'Air Force Falcons']

,sport,game_id,start_time,bookmaker,market,team,odds,point,home_team,away_team,vig_prob,fair_prb,avg_fair_prb
44,CFB,53287adc308e4bef3ab38d3652713c14,2025-11-22 18:00:00 CST,DraftKings,spreads,Air Force Falcons,1.89,3.5,Air Force Falcons,New Mexico Lobos,0.529101,0.5052,0.503867
45,CFB,53287adc308e4bef3ab38d3652713c14,2025-11-22 18:00:00 CST,FanDuel,spreads,Air Force Falcons,1.91,3.5,Air Force Falcons,New Mexico Lobos,0.523560,0.5,0.503867
46,CFB,53287adc308e4bef3ab38d3652713c14,2025-11-22 18:00:00 CST,Pinnacle,spreads,Air Force Falcons,1.92,3.5,Air Force Falcons,New Mexico Lobos,0.520833,0.5064,0.503867
